In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
from tkcalendar import DateEntry
import mysql.connector


def connect_to_db():
    try:
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="775299@Ss",  
            database="student_dbbb"
        )
        return mydb
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Error connecting to MySQL: {err}")
        return None


def create_database_and_tables():
    try:
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="775299@Ss"  # Replace with your MySQL password
        )
        cursor = mydb.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS student_dbb")
        cursor.execute("USE student_dbb")
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS students (
                student_id INT AUTO_INCREMENT PRIMARY KEY,
                student_name VARCHAR(255) NOT NULL,
                session_year VARCHAR(10) NOT NULL,
                class_section VARCHAR(5) NOT NULL
            )
        """)
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS attendance (
                attendance_id INT AUTO_INCREMENT PRIMARY KEY,
                student_id INT NOT NULL,
                attendance_date DATE NOT NULL,
                status VARCHAR(10) NOT NULL,
                session_year VARCHAR(10) NOT NULL,
                class_section VARCHAR(5) NOT NULL,
                FOREIGN KEY (student_id) REFERENCES students(student_id)
            )
        """)
        mydb.close()
        messagebox.showinfo("Success", "Database and tables created successfully.")
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Error creating database and tables: {err}")


def add_student():
    student_name = student_name_entry.get().strip()
    if not student_name:
        messagebox.showwarning("Warning", "Please enter a student name.")
        return
    
    try:
        cursor = mydb.cursor()
        sql = "INSERT INTO students (student_name, session_year, class_section) VALUES (%s, %s, %s)"
        val = (student_name, session_var.get(), class_section_var.get())
        cursor.execute(sql, val)
        mydb.commit()
        messagebox.showinfo("Success", "Student added successfully.")
        student_name_entry.delete(0, tk.END)
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Error adding student: {err}")


def on_session_select(event):
    selected_session = session_var.get()
    messagebox.showinfo("Session Selected", f"You selected session: {selected_session}")


def on_class_section_select(event):
    selected_class_section = class_section_var.get()
    messagebox.showinfo("Class Section Selected", f"You selected class section: {selected_class_section}")



def mark_attendance():
    student_id = student_id_entry.get().strip()
    status = attendance_status.get()
    attendance_date = attendance_date_entry.get_date()

    if not student_id:
        messagebox.showwarning("Warning", "Please enter a student ID.")
        return
    if status not in ['Present', 'Absent']:
        messagebox.showwarning("Warning", "Please select attendance status.")
        return
    
    try:
        cursor = mydb.cursor()
        sql = "INSERT INTO attendance (student_id, attendance_date, status, session_year, class_section) VALUES (%s, %s, %s, %s, %s)"
        val = (student_id, attendance_date, status, session_var.get(), class_section_var.get())
        cursor.execute(sql, val)
        mydb.commit()
        messagebox.showinfo("Success", "Attendance marked successfully.")
        student_id_entry.delete(0, tk.END)
        attendance_status.set('')
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Error marking attendance: {err}")


def view_attendance():
    try:
        cursor = mydb.cursor()
        sql = """
        SELECT s.student_name, a.attendance_date, a.status
        FROM attendance a
        JOIN students s ON a.student_id = s.student_id
        WHERE a.session_year = %s AND a.class_section = %s
        """
        val = (session_var.get(), class_section_var.get())
        cursor.execute(sql, val)
        result = cursor.fetchall()
        attendance_text.delete(1.0, tk.END)
        for row in result:
            attendance_text.insert(tk.END, f"Student Name: {row[0]}\nDate: {row[1]}\nStatus: {row[2]}\n\n")
    except mysql.connector.Error as err:
        messagebox.showerror("Error", f"Error fetching attendance: {err}")


def validate_login(username, password):
    return username == "shubham" and password == "123"  


def on_login():
    username = username_entry.get()
    password = password_entry.get()
    if validate_login(username, password):
        show_main_window()
        login_window.withdraw()
    else:
        messagebox.showerror("Login Failed", "Invalid username or password")


def show_main_window():
    root.deiconify()

# Initialize Tkinter
root = tk.Tk()
root.title("Student Attendance Management")
root.withdraw()


mydb = connect_to_db()
if not mydb:
    messagebox.showerror("Error", "Database connection failed. Exiting...")
    root.destroy()
    exit()

create_database_and_tables()

add_frame = tk.Frame(root, padx=20, pady=20)
add_frame.grid(row=0, column=0, padx=10, pady=10)

tk.Label(add_frame, text="Add Student").grid(row=0, column=0, columnspan=2)
tk.Label(add_frame, text="Student Name:").grid(row=1, column=0)
student_name_entry = tk.Entry(add_frame, width=30)
student_name_entry.grid(row=1, column=1)
add_button = tk.Button(add_frame, text="Add Student", command=add_student)
add_button.grid(row=2, column=0, columnspan=2, pady=10)

attendance_frame = tk.Frame(root, padx=20, pady=20)
attendance_frame.grid(row=0, column=1, padx=10, pady=10)

tk.Label(attendance_frame, text="Mark Attendance").grid(row=0, column=0, columnspan=2)
tk.Label(attendance_frame, text="Student ID:").grid(row=1, column=0)
student_id_entry = tk.Entry(attendance_frame, width=30)
student_id_entry.grid(row=1, column=1)

tk.Label(attendance_frame, text="Attendance Status:").grid(row=2, column=0)
attendance_status = tk.StringVar()
status_dropdown = tk.OptionMenu(attendance_frame, attendance_status, 'Present', 'Absent')
status_dropdown.grid(row=2, column=1)

tk.Label(attendance_frame, text="Select Session (Year):").grid(row=3, column=0)
sessions = ["2021-22", "2022-23", "2023-24", "2024-25"]  
session_var = tk.StringVar(root)
session_dropdown = ttk.Combobox(attendance_frame, textvariable=session_var, values=sessions, state="readonly")
session_dropdown.grid(row=3, column=1)
session_dropdown.bind("<<ComboboxSelected>>", on_session_select)

tk.Label(attendance_frame, text="Select Class Section:").grid(row=4, column=0)
class_sections = ["A", "B", "C"]  
class_section_var = tk.StringVar(root)
class_section_dropdown = ttk.Combobox(attendance_frame, textvariable=class_section_var, values=class_sections, state="readonly")
class_section_dropdown.grid(row=4, column=1)
class_section_dropdown.bind("<<ComboboxSelected>>", on_class_section_select)

tk.Label(attendance_frame, text="Select Attendance Date:").grid(row=5, column=0)
attendance_date_entry = DateEntry(attendance_frame, width=12, background='darkblue', foreground='white', borderwidth=2)
attendance_date_entry.grid(row=5, column=1)

mark_button = tk.Button(attendance_frame, text="Mark Attendance", command=mark_attendance)
mark_button.grid(row=6, column=0, columnspan=2, pady=10)

# GUI Layout for viewing attendance
view_frame = tk.Frame(root, padx=20, pady=20)
view_frame.grid(row=1, column=0, columnspan=2, padx=10, pady=10)

tk.Label(view_frame, text="View Attendance").grid(row=0, column=0, columnspan=2)
attendance_text = tk.Text(view_frame, width=50, height=10)
attendance_text.grid(row=1, column=0, columnspan=2)

view_button = tk.Button(view_frame, text="View Attendance", command=view_attendance)
view_button.grid(row=2, column=0, columnspan=2, pady=10)

# Login window setup
login_window = tk.Toplevel(root)
login_window.title("Login")

tk.Label(login_window, text="Username").grid(row=0, column=0, padx=10, pady=10)
username_entry = tk.Entry(login_window)
username_entry.grid(row=0, column=1, padx=10, pady=10)

tk.Label(login_window, text="Password").grid(row=1, column=0, padx=10, pady=10)
password_entry = tk.Entry(login_window, show='*')
password_entry.grid(row=1, column=1, padx=10, pady=10)

login_button = tk.Button(login_window, text="Login", command=on_login)
login_button.grid(row=2, column=0, columnspan=2, pady=10)

login_window.mainloop()

if mydb:
    mydb.close()
